<a href="https://colab.research.google.com/github/CharlieLaughton/colabtools/blob/master/NaClWE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
print(sys.version)
!wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.10.3-Linux-x86_64.sh
!bash Miniconda3-py*.sh -bfp /usr/local
!conda config --set always_yes yes
!conda config --add channels conda-forge
!conda create -n openmm python=3.7 cudatoolkit=10.0 git jupyterlab numpy pandas scipy matplotlib ipympl rdkit openbabel openmm mdtraj pymbar pdbfixer parmed openff-toolkit openmoltools openmmforcefields
sys.path.append('/usr/local/envs/openmm/lib/python3.7/site-packages')
import openmm.testInstallation
openmm.testInstallation.main()

3.7.13 (default, Mar 16 2022, 17:37:17) 
[GCC 7.5.0]
--2022-04-06 16:39:57--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.10.3-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89026327 (85M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.10.3-Linux-x86_64.sh’

Miniconda3-py37_4.1 100%[===================>]  84.90M   153MB/s    in 0.6s    

2022-04-06 16:39:58 (153 MB/s) - ‘Miniconda3-py37_4.10.3-Linux-x86_64.sh’ saved [89026327/89026327]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: - \ | done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - _openmp_mutex==4.5=1_gnu
    - brotlipy==0.7.0=py37h27cfd23_1003
    - ca-certificates==2021.7.5=h06a4308_1
    - certifi==2021.5.30=py37h

In [4]:
!ls drive/MyDrive/Colab\ Notebooks

bstate.ncrst  nacl.parm7  Untitled0.ipynb  Untitled1.ipynb


In [5]:
import mdtraj as mdt
import numpy as np
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
from sys import stdout

In [6]:
prmtop = AmberPrmtopFile('drive/MyDrive/Colab Notebooks/nacl.parm7')
inpcrd = AmberInpcrdFile('drive/MyDrive/Colab Notebooks/bstate.ncrst')
system = prmtop.createSystem(nonbondedMethod=PME, nonbondedCutoff=1*nanometer,
        constraints=HBonds)
integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)
simulation = Simulation(prmtop.topology, system, integrator)
simulation.context.setPositions(inpcrd.positions)
if inpcrd.boxVectors is not None:
    simulation.context.setPeriodicBoxVectors(*inpcrd.boxVectors)

In [7]:
class Walker(object):
    def __init__(self, coordinates, boxvectors, weight):
        self.coordinates = coordinates
        self.boxvectors = boxvectors
        self.weight = weight
        self.pc = None
        self.bin = None
        
    def copy(self):
        new = Walker(self.coordinates, self.boxvectors, self.weight)
        new.pc = self.pc
        new.bin = self.bin
        return new

class Stepper(object):
    def __init__(self, simulation, nsteps):
        self.simulation = simulation
        self.nsteps = nsteps
        
    def run(self, walkers):
        new_walkers = []
        for w in walkers:
            self.simulation.context.setPositions(w.coordinates)
            self.simulation.context.setPeriodicBoxVectors(*w.boxvectors)
            self.simulation.step(self.nsteps)
            state = self.simulation.context.getState(getPositions=True, enforcePeriodicBox=True)
            new_walkers.append(Walker(state.getPositions(), state.getPeriodicBoxVectors(), w.weight))
            
        return new_walkers

In [8]:
stepper = Stepper(simulation, 1000)
state = simulation.context.getState(getPositions=True)

In [9]:
n_reps = 5
initial_pos = state.getPositions()
initial_vecs = state.getPeriodicBoxVectors()
weight = 1.0 / n_reps
walkers = [Walker(initial_pos, initial_vecs, weight) for i in range(n_reps)]

In [10]:
new_walkers = stepper.run(walkers)

In [11]:
class ProgressCoordinator(object):
    def __init__(self, a1, a2):
        self.a1 = a1
        self.a2 = a2
        
    def run(self, walkers):
        if not isinstance(walkers, list):
            walkers = [walkers]
        for i, w in enumerate(walkers):
            crds = w.coordinates
            x1 = np.array(crds[self.a1] / nanometer)
            x2 = np.array(crds[self.a2] / nanometer)
            pc = np.sqrt(((x1-x2)*(x1-x2)).sum())
            walkers[i].pc = pc
        return walkers
    
class Binner(object):
    def __init__(self, edges):
        self.edges = edges
    
    def run(self, walkers):
        if not isinstance(walkers, list):
            walkers = [walkers]
        pcs = [w.pc for w in walkers]
        if None in pcs:
            raise TypeError('Error: missing progress coordinates...')
        bin_ids = np.digitize(pcs, self.edges)
        for i, bin_id in enumerate(bin_ids):
            walkers[i].bin = bin_id
        return walkers
    
class Recycler(object):
    def __init__(self, initial_coordinates, initial_boxvectors, target_pc, retrograde=False):
        self.initial_coordinates = initial_coordinates
        self.initial_boxvectors = initial_boxvectors
        self.target_pc = target_pc
        self.retrograde = retrograde
        
    def run(self, walkers):
        flux = 0.0
        if not isinstance(walkers, list):
            walkers = [walkers]
        for i in range(len(walkers)):
            if walkers[i].pc is None:
                raise TypeError('Error - missing progress coordinate')
            if ((not self.retrograde and walkers[i].pc > self.target_pc) or
                (self.retrograde and walkers[i].pc < self.target_pc)):
                walkers[i].coordinates = self.initial_coordinates
                walkers[i].boxvectors = self.initial_boxvectors
                walkers[i].pc = None
                walkers[i].bin = None
                flux += walkers[i].weight 
        return walkers, flux

class Bin(object):
    def __init__(self, index):
        self.index = index
        self.walkers = []
        
    def add(self, walkers):
        if not isinstance(walkers, list):
            walkers = [walkers]
        self.walkers += walkers
        
    def split_merge(self, target_size):
        if len(self.walkers) == target_size:
            return
        probs = np.array([w.weight for w in self.walkers])
        old_weight = probs.sum()
        ids = np.random.choice(range(len(self.walkers)), target_size, p=probs/old_weight)
        new_walkers = []
        for i in ids:
            new_walkers.append(self.walkers[i].copy())
        new_weight = np.array([w.weight for w in new_walkers]).sum()
        fac = old_weight / new_weight
        for i in range(len(new_walkers)):
            new_walkers[i].weight *= fac
        self.walkers = list(new_walkers)
        
class SplitMerger(object):
    def __init__(self, target_size):
        self.target_size = target_size
    
    def run(self, walkers):
        if not isinstance(walkers, list):
            walkers = [walkers]
            
        bins = {}
        for w in walkers:
            if not w.bin in bins:
                bins[w.bin] = Bin(w.bin)
            bins[w.bin].add(w)
        
        for bin in bins:
            bins[bin].split_merge(self.target_size)
        
        new_walkers = []
        for bin in bins:
            new_walkers += bins[bin].walkers
            
        return new_walkers

In [12]:
pc = ProgressCoordinator(0, 1)

In [13]:
new_walkers = pc.run(new_walkers)
splitmerger = SplitMerger(n_reps)
binner = Binner([0, 0.26, 0.28, 0.3, 0.32, 0.34, 0.36, 0.38, 0.4, 0.45, 0.5, 0.55, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5])

In [14]:
new_walkers = binner.run(new_walkers)
recycler = Recycler(initial_pos, initial_vecs, 0.26, retrograde=True)
new_walkers, flux = recycler.run(new_walkers)
print(flux)

0.0


In [15]:
new_walkers = splitmerger.run(new_walkers)
print([(w.bin, w.weight) for w in new_walkers])

[(18, 0.08000000000000002), (18, 0.08000000000000002), (18, 0.08000000000000002), (18, 0.08000000000000002), (18, 0.08000000000000002), (20, 0.04000000000000001), (20, 0.04000000000000001), (20, 0.04000000000000001), (20, 0.04000000000000001), (20, 0.04000000000000001), (19, 0.04000000000000001), (19, 0.04000000000000001), (19, 0.04000000000000001), (19, 0.04000000000000001), (19, 0.04000000000000001), (17, 0.04000000000000001), (17, 0.04000000000000001), (17, 0.04000000000000001), (17, 0.04000000000000001), (17, 0.04000000000000001)]


In [16]:
n_cycles=5
for i in range(n_cycles):
    new_walkers = stepper.run(new_walkers)
    new_walkers = pc.run(new_walkers)
    new_walkers = binner.run(new_walkers)
    new_walkers, flux = recycler.run(new_walkers)
    if flux > 0.0:
        new_walkers = pc.run(new_walkers)
        new_walkers = binner.run(new_walkers)
    print(i, flux, len(new_walkers), min([w.pc for w in new_walkers]))
    new_walkers = splitmerger.run(new_walkers)

0 0.0 20 1.0192865616249116
1 0.0 30 0.8696518114140716
2 0.0 35 0.8721983462618329
3 0.0 40 0.7423955191941667
4 0.0 45 0.5975007798455428


In [17]:
for i in range(n_cycles):
    new_walkers = stepper.run(new_walkers)
    new_walkers = pc.run(new_walkers)
    new_walkers = binner.run(new_walkers)
    new_walkers, flux = recycler.run(new_walkers)
    if flux > 0.0:
        new_walkers = pc.run(new_walkers)
        new_walkers = binner.run(new_walkers)
    print(i, flux, len(new_walkers), min([w.pc for w in new_walkers]))
    new_walkers = splitmerger.run(new_walkers)

0 0.0 55 0.41547853413319286
1 0.0 55 0.4386568624383967
2 0.0 65 0.4293520778973081
3 0.0 70 0.41541203315929903
4 0.0 70 0.44347474000409803


In [18]:
for i in range(n_cycles):
    new_walkers = stepper.run(new_walkers)
    new_walkers = pc.run(new_walkers)
    new_walkers = binner.run(new_walkers)
    new_walkers, flux = recycler.run(new_walkers)
    if flux > 0.0:
        new_walkers = pc.run(new_walkers)
        new_walkers = binner.run(new_walkers)
    print(i, flux, len(new_walkers), min([w.pc for w in new_walkers]))
    new_walkers = splitmerger.run(new_walkers)

0 0.0 70 0.42011142374537863
1 0.0 65 0.42210645377486006
2 0.0 70 0.3730744829512078
3 2.7534262042360994e-06 75 0.4381875392282308
4 0.0 65 0.41776101233710317
